# 概要

すべてのレイヤを直列に（シーケンシャルに）接続するモデルはSequential APIで簡単に実現できる。<br>
一方それ以上に複雑なモデルは対応できないため関数型APIを使ってニューラルネットワークを構築する。

# ワイド・アンド・ディープニューラルネットワーク

シーケンシャルなニューラルネットワークではすべてのデータに全層の強制する。<br>
そのためデータに含まれる単純なパターンが層を通過する中で歪められてしまうことがある。<br>
そこで層の通過をスキップする"ワイド"パスを含むモデルを作成する。

今回作成するモデルは以下：

![ワイドアンドディープニューラルネット](./fig/wide_and_deep_nural_net.png)

## パッケージインポート

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

print(f'tensorflow ver.{tf.__version__}')
print(f'keras ver.{keras.__version__}')

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tensorflow ver.2.5.0
keras ver.2.5.0


## データロードと前処理

###  データロード（sklearn.datasets.fetch_california_housing）

In [3]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

### validation分割

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

# データサイズを確認
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_valid.shape : {x_valid.shape}')
print(f'y_valid.shape : {y_valid.shape}')

x_train.shape : (11610, 8)
y_train.shape : (11610,)
x_valid.shape : (3870, 8)
y_valid.shape : (3870,)


trainデータを確認

In [5]:
pd_x_train = pd.DataFrame(x_train, columns=housing.feature_names)
display(pd_x_train)
pd_x_train.info()
pd_x_train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.2984,50.0,5.781690,1.133803,713.0,2.510563,37.78,-122.18
1,4.0417,32.0,5.351515,0.975758,876.0,2.654545,33.81,-117.85
2,4.6452,45.0,5.689139,0.985019,681.0,2.550562,33.79,-118.14
3,4.0174,28.0,5.028056,1.068136,2846.0,2.851703,37.28,-121.96
4,6.8939,38.0,5.062069,1.039080,833.0,1.914943,37.76,-122.44
...,...,...,...,...,...,...,...,...
11605,4.1724,14.0,5.239496,1.080672,3568.0,2.998319,34.32,-118.44
11606,4.2222,28.0,6.304478,1.083582,1011.0,3.017910,36.54,-119.48
11607,2.5474,13.0,8.213768,2.177536,756.0,2.739130,35.47,-118.61
11608,3.8365,29.0,5.503686,1.122850,1187.0,2.916462,34.14,-117.90


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11610 entries, 0 to 11609
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      11610 non-null  float64
 1   HouseAge    11610 non-null  float64
 2   AveRooms    11610 non-null  float64
 3   AveBedrms   11610 non-null  float64
 4   Population  11610 non-null  float64
 5   AveOccup    11610 non-null  float64
 6   Latitude    11610 non-null  float64
 7   Longitude   11610 non-null  float64
dtypes: float64(8)
memory usage: 725.8 KB


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000
mean,3.864953,28.705771,5.426279,1.096556,1416.647373,3.117047,35.624672,-119.559562
std,1.873837,12.610790,2.476206,0.446533,1148.337199,12.664496,2.142690,2.005369
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.550000,-124.350000
25%,2.568200,18.000000,4.442249,1.005900,783.000000,2.422020,33.930000,-121.790000
50%,3.540350,29.000000,5.228400,1.049202,1159.000000,2.815672,34.250000,-118.480000
75%,4.723700,37.000000,6.047265,1.099194,1707.750000,3.279769,37.710000,-117.990000
max,15.000100,52.000000,141.909091,25.636364,35682.000000,1243.333333,41.950000,-114.310000


### 前処理

#### スケーリング

### [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
データの標準化を行う。

代表的なメソッドは以下：

|メソッド|説明|
|---|---|
|fit()|標準化するための平均と分散を計算する。|
|trasform()|（事前に計算した平均と分散を使用して）標準化を行う。|
|fit_transform()|平均と分散を計算し、標準化を行う。|

In [6]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)  # x_trainの平均・分散を使用する（のはなぜ？）
x_test = scaler.transform(x_test)  # x_trainの平均・分散を使用する（のはなぜ？）

#### inputデータ分割

In [7]:
# ワイドパス、ディープパス用で分割
x_train_A, x_train_B = x_train[:, :5], x_train[:, 2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A, x_test_B = x_test[:, :5], x_test[:, 2:]

# モデル作成

## レイヤ構成を定義

Sequential APIでは層の順番を定義したが、
関数型APIではより複雑に層同士の接続方法を定義することが出来る。<br>

具体的には層の出力を別の層の入力に渡すことで接続方法をモデルに指示する。<br>
これがレイヤクラスのインスタンスを生成する際に入力を渡す姿が関数のように見えるため関数型APIと言われる所以である。

ちなみにインスタンス生成時は層の接続方法を指示しているだけで、データはまだ処理されていない。

In [8]:
# 初期化時にレイヤのリストを渡すことでレイヤ定義も同時に行う
input_A = keras.layers.Input(shape=[5], name='wide_input')  # ワイドパスのインプット層
input_B = keras.layers.Input(shape=[6], name='deep_input')  # ディープパスのインプット層
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_B)  # input_B → hidden_1 へ
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)  # hidden_1 → hidden_2 へ
concat = keras.layers.concatenate([input_A, hidden2])  # input_Aとhidden_2を結合
output = keras.layers.Dense(1, name='output')(concat)  # concat →　output

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

## モデルのコンパイル

In [9]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
______________________________________________________________________________________________

# 学習と評価

### コールバック設定

In [10]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [11]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [12]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

## 学習

In [13]:
history = model.fit((x_train_A, x_train_B), y_train, epochs=20, validation_data=((x_valid_A, x_valid_B), y_valid), callbacks=[tensorboard_cb, early_stopping_cb, print_valid_train_ration_cb])

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 1.8493 - val_loss: 0.8307

val/train: 0.45
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.7744 - val_loss: 0.7046

val/train: 0.91
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6810 - val_loss: 0.6628

val/train: 0.97
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6448 - val_loss: 0.6359

val/train: 0.99
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6185 - val_loss: 0.6113

val/train: 0.99
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5963 - val_loss: 0.5919

val/train: 0.99
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5769 - val_loss: 0.5737

val/train: 0.99
Epoch 8/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5593 - val_loss: 0.5574

val/train: 1.00
Epoch 9/20
363/363 [==============================] - 1s 3ms/ste

## 学習結果の可視化

In [14]:
%load_ext tensorboard
%tensorboard --logdir ./my_logs

Reusing TensorBoard on port 6006 (pid 29808), started 1:22:21 ago. (Use '!kill 29808' to kill it.)

## 学習結果の評価

In [16]:
mse_test = model.evaluate((x_test_A, x_test_B), y_test)
mse_test

162/162 [==============================] - 0s 2ms/step - loss: 0.4713


0.47130921483039856

# 学習済みモデルを使った予測

In [17]:
# サンプル用にデータサイズを限定
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

y_pred = model.predict((x_new_A, x_new_B))
print(f'predict : {y_pred.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict : [1.7249901 3.5990105 2.1218798]
correct : [0.945 4.772 1.523]


# 複数の出力をもつニューラルネットワーク

マルチタスク分類に対応したモデルを作成する場合、出力を複数用意する必要がある。<br>
例えば同じ顔写真から表情と眼鏡を掛けているかどうかを分類するような問題がこれに当たる。

また、ユースケースとしては正則化テクニックとして使用する場合がある。<br>
例えばネットワークの下位層だけで独自に役立つ情報が学習ができるように補助出力を追加するような場合である。

今回は正則化テクニックを想定して、下位の隠れ層2層だけで独自に学習が出来るニューラルネットを作成する。

![複数出力をもつニューラルネット](./fig/multiple_outputs_nural_net.png)


モジュールインポートからデータ前処理まではワイド・アンド・ディープニューラルネットを流用する

# モデル作成

## レイヤ構成を定義

In [18]:
# 初期化時にレイヤのリストを渡すことでレイヤ定義も同時に行う
input_A = keras.layers.Input(shape=[5], name='wide_input')  # ワイドパスのインプット層
input_B = keras.layers.Input(shape=[6], name='deep_input')  # ディープパスのインプット層
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_B)  # input_B → hidden_1 へ
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)  # hidden_1 → hidden_2 へ
concat = keras.layers.concatenate([input_A, hidden2])  # input_Aとhidden_2を結合
output = keras.layers.Dense(1, name='output')(concat)  # concat →　output
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)  # hidden_2 →　aux_output

multi_outputs_model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

## モデルのコンパイル

個々の出力に対して専用の損失関数を準備する必要があるため損失関数のリストを引数に渡す。<br>
またデフォルトでは単純に個々の損失を合計してモデル全体の損失とする。重みづけする場合はloss_weightsで重みを指定する。<br>
今回の例では正則化のための補助出力よりもメイン出力の方が重要なのでメイン出力の重みを大きくする。

In [19]:
multi_outputs_model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD())

multi_outputs_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           930         dense_2[0][0]                    
____________________________________________________________________________________________

# 学習と評価

## 学習

学習時もそれぞれの出力に対してラベルを指定する必要がある。<br>
今回の例では両方同じものを予測しているので同じラベルを渡す。

### コールバック設定

In [20]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [21]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [22]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

In [23]:
history = multi_outputs_model.fit([x_train_A, x_train_B], [y_train, y_train], epochs=20, validation_data=([x_valid_A, x_valid_B], [y_valid, y_valid]), callbacks=[tensorboard_cb, early_stopping_cb, print_valid_train_ration_cb])

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 0.8968 - output_loss: 0.7854 - aux_output_loss: 1.8993 - val_loss: 0.6066 - val_output_loss: 0.5483 - val_aux_output_loss: 1.1310

val/train: 0.68
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5538 - output_loss: 0.4991 - aux_output_loss: 1.0461 - val_loss: 0.5622 - val_output_loss: 0.5120 - val_aux_output_loss: 1.0141

val/train: 1.02
Epoch 3/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5218 - output_loss: 0.4800 - aux_output_loss: 0.8984 - val_loss: 0.5126 - val_output_loss: 0.4719 - val_aux_output_loss: 0.8793

val/train: 0.98
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4999 - output_loss: 0.4649 - aux_output_loss: 0.8149 - val_loss: 0.4910 - val_output_loss: 0.4563 - val_aux_output_loss: 0.8032

val/train: 0.98
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4824 - output_loss: 0.4519 - aux_output_

## 学習結果の可視化

In [24]:
%load_ext tensorboard
%tensorboard --logdir ./my_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 29808), started 1:22:47 ago. (Use '!kill 29808' to kill it.)

## 学習結果の評価

In [26]:
total_loss, main_loss, aux_loss = multi_outputs_model.evaluate((x_test_A, x_test_B), (y_test, y_test))
total_loss, main_loss, aux_loss

162/162 [==============================] - 0s 2ms/step - loss: 0.3672 - output_loss: 0.3512 - aux_output_loss: 0.5114


(0.36717718839645386, 0.3511541485786438, 0.5113843679428101)

# 学習済みモデルを使った予測

In [27]:
# サンプル用にデータサイズを限定
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

y_pred_main, y_pred_aux = multi_outputs_model.predict((x_new_A, x_new_B))
print(f'predict_main : {y_pred_main.reshape(-1)}')
print(f'predict_aux : {y_pred_aux.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict_main : [1.3989782 3.9601068 1.690203 ]
predict_aux : [2.0541828 3.8567936 1.4224538]
correct : [0.945 4.772 1.523]
